Take Raw LIGO O3b data, inject 30 seconds long signals

In [ ]:
import numpy as np
import h5py
import os
import torch
import matplotlib.pyplot as plt

from tqdm import tqdm

from dataset import Spectrogram
from torch.utils.data import Dataset

ModuleNotFoundError: No module named 'torchaudio'

In [2]:
num_data = 10000

duration = 8
sample_rate=2048

In [3]:
# Paths to data
data_path = '../data/'
in_path = data_path + 'raw/train/background/'
out_path = data_path+'chunks/'

train_data = os.listdir(in_path)
train_data = [data for data in train_data if data.endswith('.hdf5')]
print(train_data)

out_data = os.listdir(out_path)
[os.remove(out_path + file) for file in out_data]

['background-1241204831-9238.hdf5', 'background-1241002600-3101.hdf5', 'background-1241310397-20000.hdf5', 'background-1241113492-10318.hdf5', 'background-1240743328-5673.hdf5', 'background-1241263169-9449.hdf5', 'background-1240939428-1116.hdf5', 'background-1241163878-9945.hdf5', 'background-1241393338-4562.hdf5', 'background-1240940621-20000.hdf5', 'background-1240720690-14972.hdf5', 'background-1241123878-20000.hdf5', 'background-1241088652-3696.hdf5', 'background-1240784297-1811.hdf5', 'background-1240919428-20000.hdf5', 'background-1240624412-20000.hdf5', 'background-1240750849-12554.hdf5', 'background-1241417182-20000.hdf5', 'background-1241143878-20000.hdf5', 'background-1241243169-20000.hdf5', 'background-1240700690-20000.hdf5', 'background-1240795605-18638.hdf5', 'background-1241302060-7884.hdf5', 'background-1240892261-14732.hdf5', 'background-1241001309-1186.hdf5', 'background-1241068861-4928.hdf5', 'background-1240960621-590.hdf5', 'background-1240594562-12186.hdf5', 'back

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [4]:
durations = np.array([int(i.split('-')[-1][:-5]) for i in train_data if i[0] != 's'])
chunks = durations//duration
chunks_per_file = (chunks*num_data*1.1//sum(chunks)).astype(int)

In [5]:
param_dict = ["chirp_mass", "mass_ratio", "chi1", "chi2", "distance", "phic", "inclination", 'dec', 'phi', 'psi', 'snr']

In [8]:
from Waveform import new_hphc
from ml4gw.transforms import SpectralDensity
from ml4gw.distributions import PowerLaw, Sine, Cosine, DeltaFunction
from torch.distributions import Uniform

device='cpu'

fftlength = 2
spectral_density = SpectralDensity(
    sample_rate=sample_rate,
    fftlength=fftlength,
    overlap=None,
    average="median",
).to(device)

index_len = duration*sample_rate

for i in tqdm(range(len(train_data))):

    print(i)

    data = h5py.File(in_path + train_data[i])
    H1 = np.array(data['H1'])
    L1 = np.array(data['L1'])

    ## get psd

    background = torch.stack([torch.from_numpy(H1), torch.from_numpy(L1)])
    psd = spectral_density(background.double())

    
    num_chunks = chunks_per_file[i]
    length = durations[i]
    ranges = np.linspace(0, length, num_chunks+1)
    times = [np.random.randint(ranges[i], ranges[i+1]-duration, 1)[0] for i in range(num_chunks)]

    waveforms = []
    parameters = {k: torch.tensor([]) for k in param_dict}

    ln = 0
    while ln < num_chunks:
        waveform, parameter, h1_snr, l1_snr = new_hphc(1, psd=psd)

        if h1_snr > 13 and l1_snr > 13:
            waveforms.append(waveform[0])

            for k, v in parameter.items():
                parameters[k] = torch.cat((parameters[k], v), dim=0)
            
            ln += 1
        

    #waveforms, parameters = new_hphc(1, psd=psd)
    waveforms = torch.stack(waveforms)

    
    
    h1 = (waveforms[:, 0]).numpy()  ## need to be projected using compute_observed_strain. Float Error
    l1 = (waveforms[:, 1]).numpy() ## Right now, I am using polarizations as projections which is wrong

    H = []
    L = []

    for j in range(num_chunks):

        index = int(times[j]*sample_rate)
        H_chunk = H1[index: int(index + index_len)]  ## needs same filtering as the waveforms
        L_chunk = L1[index: int(index + index_len)]

        H_chunk = H_chunk + h1[j]
        L_chunk = L_chunk + l1[j]

        H.append(H_chunk)
        L.append(L_chunk)

    H = np.vstack(H)
    L = np.vstack(L)

    


    out = h5py.File(out_path+train_data[i], 'w')
    out.create_dataset('H1', data=H)
    out.create_dataset('L1', data=L)

    for k, v in parameters.items():
        out.create_dataset(k, data=v)

    out.close()

  0%|                                                    | 0/43 [00:00<?, ?it/s]

0


  2%|█                                           | 1/43 [00:07<05:31,  7.89s/it]

1


  5%|██                                          | 2/43 [00:10<03:07,  4.57s/it]

...

 95%|█████████████████████████████████████████  | 41/43 [06:53<00:15,  7.68s/it]

41


 98%|██████████████████████████████████████████ | 42/43 [06:59<00:06,  6.99s/it]

42


100%|███████████████████████████████████████████| 43/43 [07:04<00:00,  9.88s/it]


### Spectrogram Loading Description

The spectrogram loading process involves the following steps:

1. **Lazy Loading of Data**: 
    - A `LazyDataset` class is used to load data paths lazily from the specified directory. This approach is GPU-friendly as it avoids loading all data into memory at once.

2. **Iterative Processing**:
    - The `compute_save_spectrograms` function iterates over the dataset starting from a specified index (`start_idx`).
    - For each file, it loads the data (`H1` and `L1` channels) and associated parameters.

3. **Spectrogram Computation**:
    - A `Spectrogram` object is used to compute spectrograms for the loaded data. The parameters are also processed alongside.

4. **Saving Results**:
    - The computed spectrograms and parameters are saved as `.pt` files in the specified directory (`data_save_dir`) using PyTorch's `torch.save` method.

This process ensures efficient handling of large datasets and prepares the data for further analysis or model training.

In [9]:
class LazyDataset(Dataset):
    """
    Class for lazy loading of data and parameters from directories. (GPU friendly)
    """
    def __init__(self, data_dir):
        # Load directories lazily e.g. just the paths
        self.data_paths = sorted([
            os.path.join(data_dir, f)
            for f in os.listdir(data_dir) if f.endswith('.pt') or f.endswith('.hdf5')
        ])
    
    def __len__(self):
        return len(self.data_paths)
    
    def __getitem__(self, idx):
        # Load data here
        data = h5py.File(self.data_paths[idx])
        return data

    def get_paths(self):
        # Return the paths for the given index
        return self.data_paths
    
def compute_save_spectrograms(data_dir, param_keys, data_save_dir):
    lazyload = LazyDataset(data_dir)
    start_idx = 0
    print(f'Number of files: {len(lazyload) - start_idx}')


    for i in tqdm(range(start_idx, len(lazyload))):
        # Load data lazily (GPU friendly)
        f = lazyload[i]
        batch = np.array([f['H1'], f['L1']])
        params = np.array([np.array(f[key]) for key in param_keys[:-1]])
        spec = Spectrogram(batch.shape[-1])
        print(batch.shape, params.shape)
        data, params = spec.forward(batch, params.T)
        print(data.shape, data.dtype, type(data))
        print(f'Attempting to write to: {data_save_dir}')
        try:
            torch.save({'data': data, 'params': params}, os.path.join(data_save_dir, f'spectrogram_{i}.pt'))
            print(f'Wrote file {i}')
        except Exception as e:
            print(f'Error writing file {i}: {e}')
            continue
        
    return

param_keys = ['chirp_mass', 'mass_ratio', 'distance', 'psi','phi', 'dec', 'snr']
data_dir = '../data/chunks/'
data_save_dir = '../data/spectrograms/'

compute_save_spectrograms(data_dir, param_keys, data_save_dir)

Number of files: 43


  0%|                                                    | 0/43 [00:00<?, ?it/s]

(2, 177, 16384) (6, 177)
torch.Size([177, 10, 2, 80, 205]) torch.float32 <class 'torch.Tensor'>
Attempting to write to: ../data/spectrograms/


  2%|█                                           | 1/43 [00:05<03:45,  5.38s/it]

Wrote file 0
(2, 277, 16384) (6, 277)
torch.Size([277, 10, 2, 80, 205]) torch.float32 <class 'torch.Tensor'>
Attempting to write to: ../data/spectrograms/


...

Wrote file 40
(2, 454, 16384) (6, 454)
torch.Size([454, 10, 2, 80, 205]) torch.float32 <class 'torch.Tensor'>
Attempting to write to: ../data/spectrograms/


 98%|██████████████████████████████████████████ | 42/43 [07:10<00:07,  7.87s/it]

Wrote file 41
(2, 150, 16384) (6, 150)
torch.Size([150, 10, 2, 80, 205]) torch.float32 <class 'torch.Tensor'>
Attempting to write to: ../data/spectrograms/


100%|███████████████████████████████████████████| 43/43 [07:14<00:00, 10.11s/it]

Wrote file 42
